## CellProfiler-OMERO demo
We have setup an example notebook to guide you through the steps to analise images stored in OMERO using CellProfiler

Let's start by importing some libraries we are going to need

In [20]:
import ezomero

import cellprofiler_core.preferences as cp_preferences
import cellprofiler_core.pipeline as cp_pipeline
import cellprofiler_core.measurement as cp_measurement
import cellprofiler_core.modules as cp_modules

import pandas as pd
import tempfile

In [21]:
# Make CellProfiler run without a GUI
cp_preferences.set_headless()

# Tell CellProfiler to save output in a temp directory
output_dir = tempfile.TemporaryDirectory()
cp_preferences.set_default_output_directory(output_dir.name)

/tmp/tmpvn012q4o


Let's connect to OMERO. When we connect we get a connection object that we will have to use in every interaction with OMERO.

In [28]:
# Creating a connection object
host = "omero.mri.cnrs.fr"
port = 4064
conn = ezomero.connect(host=host, port=port)

# Connecting
conn.connect()
# The connection will timeout after a period of inactivity. To avoid that, we can tell the connection to refresh the connection every 60 seconds
conn.c.enableKeepAlive(60)
# Let's verify that we are connected
conn.isConnected()

False


In [25]:
# remove the output directory
output_dir.cleanup()

# and close the connection to the OMERO server
conn.close()